In [ ]:
import numpy as np
import pandas as pd
from astropy.stats import mad_std, sigma_clip
import re
import matplotlib.pyplot as plt
import holoviews as hv
import hvplot.pandas
from cytoolz import get_in
from astropy.time import Time
from datetime import datetime, timedelta
from pathlib import Path
import pickle

IDX = pd.IndexSlice

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.projects.sigma_circuits.segmentation as segmentation
import paulssonlab.projects.sigma_circuits.matriarch_stub as matriarch_stub
import paulssonlab.projects.sigma_circuits.experiment as experiment
import paulssonlab.io.metadata as metadata

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
hv.extension("bokeh")

# Load data

In [ ]:
# nd2_filename = "/n/scratch3/groups/hms/sysbio/paulsson/jqs1/210324/PWM_flipped.nd2"
# nd2_filename2 = "/n/scratch3/groups/hms/sysbio/paulsson/jqs1/210204/psc101_bistable.nd2"

In [ ]:
# nd2 = matriarch_stub.get_nd2_reader(nd2_filename)

In [ ]:
# filename = "/n/groups/paulsson/jqs1/sigma-circuits/210329_expt210324flipped.pickle"
# filename = "/n/groups/paulsson/jqs1/sigma-circuits/210329_expt210204nonflipped_psc101.pickle"
filename = "/n/groups/paulsson/jqs1/sigma-circuits/210402_expt210331ramp.pickle"
data = pickle.load(open(filename, "rb"))
df = data["table"]
md = data["metadata"]
mux_log = data["mux_log"]
experiment_txt = data["experiment.txt"]
grid_df = data["grid"]

# Acquisition times

In [ ]:
md.keys()

In [ ]:
dtimeabsolute = get_in(
    ("SLxPictureMetadata", "dTimeAbsolute"),
    md["image_metadata_sequence"],
)
dtimeabsolute = Time(dtimeabsolute, format="jd").to_datetime()
dtimemsec = get_in(
    ("SLxPictureMetadata", "dTimeMSec"),
    md["image_metadata_sequence"],
)
tdelta = timedelta(milliseconds=dtimemsec)

In [ ]:
tdelta.total_seconds()

In [ ]:
first_position_times = md["acquisition_times"][:: len(grid_df)] / 1e3  # seconds

In [ ]:
last_good_cycle = np.argmax(np.diff(first_position_times) == 0) - 1

In [ ]:
seconds_per_timepoint = first_position_times[last_good_cycle] / last_good_cycle

In [ ]:
mux_times = [m[0] for m in mux_log]

In [ ]:
mux_timepoints = [
    (m - dtimeabsolute).total_seconds() / seconds_per_timepoint for m in mux_times
]

In [ ]:
mux_timepoints

In [ ]:
mux_timepoints2 = [
    np.argmax((m - dtimeabsolute).total_seconds() <= first_position_times)
    for m in mux_times
]

In [ ]:
mux_timepoints2

In [ ]:
plt.plot(md["acquisition_times"][:5000])

# Filtered

In [ ]:
%%time
cells = df.join(grid_df).reset_index()
filtered_cells = cells[
    cells["area"].between(200, 800)
    & cells["GFP-PENTA"].between(100, 4000)
    & ~cells["row"].isin(["F"])
][["GFP-PENTA", "row", "t"]]

filtered_cells2 = filtered_cells.groupby(["row", "t"]).apply(
    lambda y: sigma_clip(y, sigma=2, maxiters=10, masked=False)
)

means = filtered_cells2.apply(np.mean)
sigmas = filtered_cells2.apply(np.std)

limits = pd.DataFrame({"lower": means - sigmas, "upper": means + sigmas})

In [ ]:
mean_plot = means.hvplot.line("t", by="row", logy=True)
noise_plot = limits.hvplot.area(
    x="t", y="lower", y2="upper", by="row", stacked=False, alpha=0.2, logy=True
)
mux_events = hv.Overlay(
    [
        hv.VLine(t).opts(alpha=0.2, color="grey", line_dash="dotted")
        for t in mux_timepoints
    ]
)

# line_alpha = 0.2
# IPTG_starts = (
#     hv.VLine(12).opts(alpha=line_alpha, color="blue")
#     * hv.VLine(60).opts(alpha=line_alpha, color="blue")
#     * hv.VLine(108).opts(alpha=line_alpha, color="blue")
#     * hv.VLine(156).opts(alpha=line_alpha, color="blue")
# )
# EZ_starts = (
#     hv.VLine(30).opts(alpha=line_alpha, color="grey")
#     * hv.VLine(78).opts(alpha=line_alpha, color="grey")
#     * hv.VLine(126).opts(alpha=line_alpha, color="grey")
#     * hv.VLine(174).opts(alpha=line_alpha, color="grey")
# )
# aTc_starts = (
#     hv.VLine(42).opts(alpha=line_alpha, color="red")
#     * hv.VLine(90).opts(alpha=line_alpha, color="red")
#     * hv.VLine(138).opts(alpha=line_alpha, color="red")
#     * hv.VLine(186).opts(alpha=line_alpha, color="red")
# )

# (IPTG_starts*EZ_starts*aTc_starts*means*errors).opts(width=1000,height=400)
# (means * errors).opts(width=1000, height=400)

In [ ]:
(mean_plot * noise_plot).opts(width=800, height=400)

In [ ]:
(mean_plot * noise_plot * mux_events).opts(width=800, height=400)

In [ ]:
print(experiment_txt)

In [ ]:
mux_log

In [ ]:
mux_timepoints